In [3]:
import re
import logging

#Set debugging level
#Levels are CRITICAL, ERROR, WARNING, INFO, and DEBUG, 
#from least to most verbose
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL) 

#Paths for input/output files
ordersFile = 'input/orders.txt'
outputFile = 'output/output.txt'

#Regular expression for properly-formatted order lines
ordersRegEx='^\s*([0-9]+)\s*,\s*([0-9]+)\s*,\s*([0-9]+)\s*,\s*\'([\w\s]+\w*)\'$'

#Current bonus promotion -- easy to change in the future
bonus =  {       'milk': {'milk': 1, 'dark': 0, 'white': 0, 'sugar free': 1},
                 'dark': {'milk': 0, 'dark': 1, 'white': 0, 'sugar free': 0},
                'white': {'milk': 0, 'dark': 0, 'white': 1, 'sugar free': 1},
           'sugar free': {'milk': 0, 'dark': 1, 'white': 0, 'sugar free': 1}}

try:
    with open(ordersFile,'r') as orders, open(outputFile,'w') as output:  
        line = orders.readline()
        cnt = 1
        while line:
            chocolates = {'milk': 0, 'dark': 0, 'white': 0, 'sugar free': 0}
            parsedLine = re.match(ordersRegEx, line)
            if parsedLine: #Lines which do not match regex will be ignored
                logger.debug("Line {}: {}".format(cnt, line.strip()))
                type_bought = parsedLine[4]
                wrappers_needed = int(parsedLine[3])
                chocolates[type_bought] = int(int(parsedLine[1])/int(parsedLine[2]))
                logger.info("Initial Purchase: {}\nWrappers needed for bonus: {}".format(chocolates,wrappers_needed))
                wrappers = dict(chocolates) #Copy chocolates to wrappers
                while any(w>=wrappers_needed for w in wrappers.values()): #Still have enough wrappers for bonus
                    for bonus_type in ([wrap_type  for (wrap_type, qty) in wrappers.items() if qty >= wrappers_needed]):
                        logger.debug("Bonus_type: {}".format(bonus_type))
                        logger.debug("Bonus: {}".format(bonus[bonus_type]))
                        wrappers[bonus_type] -= wrappers_needed #Remove redeemed wrappers from stash
                        for choc_type in bonus[bonus_type]:
                            chocolates[choc_type] += bonus[bonus_type][choc_type]
                            wrappers[choc_type] += bonus[bonus_type][choc_type] 
                        logger.debug("New Chocolates: {}".format(chocolates))
                        logger.debug("New Wrappers: {}".format(wrappers))
                # Not enough wrappers left to exchange
                logger.debug("Final Wrappers: {}".format(wrappers))
                logger.info("Final Chocolates: {}".format(chocolates))
                output.write('milk {c[milk]}, dark {c[dark]}, white {c[white]}, sugar free {c[sugar free]}\n'.format(c=chocolates))
            else:
                logger.info("Line {}: {}".format(cnt, "Rejected"))
            line = orders.readline()
            cnt += 1
    print("Output written to {}.".format(outputFile))
except:
    logger.critical("File error!  Please check paths for input and output files.")
        

Output written to output/output.txt.
